[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/ada-k/GANSinAction-TheLangrBook/blob/main/gans_mnistGeneration.ipynb)

In [ ]:
# cup of tea
%matplotlib inline

import matplotlib.pyplot as plt
from keras.layers import Activation, Dense, BatchNormalization, Dropout, Flatten, Reshape
from keras.models import Sequential
from keras.optimizers import Adam
from keras.datasets import mnist
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import Conv2D, Conv2DTranspose
import numpy as np

In [ ]:
# input dimensions
img_shape = (28, 28, 1)
z_dim = 100

In [ ]:
# generator definition: transposed convolution
def build_generator(z_dim):
    model = Sequential()

    # Take a random noise vector and reshape it into a 7 × 7 × 256 tensor through afully connected layer
    model.add(Dense(256 * 7 * 7, input_dim = z_dim))
    model.add(Reshape((7, 7, 256)))

    # Use transposed convolution, transforming the 7 × 7 × 256 tensor into a 14 × 14× 128 tensor
    model.add(Conv2DTranspose(128, kernel_size = 3, strides = 2, padding = 'same'))

    #batch normalisation and leakyrelu activation
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha = 0.01))

    # transforming the 14 × 14 × 128 tensor into a 14 ×14 × 64 tensor (h and w remain unchanged by setting stride = 1)
    model.add(Conv2DTranspose(64, kernel_size = 3, strides = 1, padding = 'same'))

    # batch normalisation and leakyrelu activation again
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha = 0.01))

    # transforming the 14 × 14 × 64 tensor into the out-put image size, 28 × 28 × 1
    model.add(Conv2DTranspose(1, kernel_size = 3, strides = 2, padding = 'same'))

    # tanh activation on output layer
    model.add(Activation('tanh'))

    return model

In [ ]:
# build discriminator
def build_discriminator(img_shape):
    model = Sequential()
    # Use a convolutional layer to transform a 28 × 28 × 1 input image into a 14 × 14 ×32 tensor
    model.add(Conv2D(32, kernel_size = 3, strides = 2, input_shape = img_shape, padding = 'same'))

    # apply the Leaky ReLU activation function
    model.add(LeakyReLU(alpha = 0.01))

    # Use a convolutional layer, transforming the 14 × 14 × 32 tensor into a 7 × 7 × 64tensor
    model.add(Conv2D(64, kernel_size = 3, strides = 2, input_shape = img_shape, padding = 'same'))

    # Apply batch normalization and the Leaky ReLU activation function
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha = 0.01))

    # Use a convolutional layer, transforming the 7 × 7 × 64 tensor into a 3 × 3 × 128tensor
    model.add(Conv2D(128, kernel_size = 3, strides = 2, input_shape = img_shape, padding = 'same'))

    # Apply batch normalization and the Leaky ReLU activation function
    model.add(BatchNormalization())
    model.add(LeakyReLU(alpha = 0.01))

    # Flatten the 3 × 3 × 128 tensor into a vector of size 3 × 3 × 128 = 1152
    model.add(Flatten())
    # output layer
    model.add(Dense(1, activation = 'sigmoid'))

    return model

In [ ]:
# build and run
def build_gan(generator, discriminator):
    model = Sequential()
    model.add(generator)
    model.add(discriminator)

    return model

# build and compile the discriminator
discriminator = build_discriminator(img_shape)
discriminator.compile(loss = 'binary_crossentropy', optimizer = Adam(), metrics = ['accuracy'])

# build generator
generator = build_generator(z_dim)

discriminator.trainable = False # keep discriminator's parameters unchangeable

gan = build_gan(generator, discriminator)
gan.compile(loss = 'binary_crossentropy', optimizer = Adam())
gan.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_8 (Sequential)    (None, 28, 28, 1)         1637121   
_________________________________________________________________
sequential_7 (Sequential)    (None, 1)                 95489     
Total params: 1,732,610
Trainable params: 1,636,737
Non-trainable params: 95,873
_________________________________________________________________


In [ ]:
# display sample images
def sample_images(generator, image_grid_rows=4, image_grid_columns=4):  
    # random noise sample  
    z = np.random.normal(0, 1, (image_grid_rows * image_grid_columns, z_dim))   
    # x* - generate images from the noise 
    gen_imgs = generator.predict(z)   
    # rescale images to [0, 1] pixel values      
    gen_imgs = 0.5 * gen_imgs + 0.5     
    # img grid              
    fig, axs = plt.subplots(image_grid_rows,                                  
                            image_grid_columns,                            
                            figsize=(4, 4),                            
                            sharey=True,                            
                            sharex=True)    
    cnt = 0
    for i in range(image_grid_rows):
        for j in range(image_grid_columns):            
            axs[i, j].imshow(gen_imgs[cnt, :, :, 0], cmap='gray')  
            plt.savefig('/content/sample_data/')  # write image to file
            plt.cla()             
            axs[i, j].axis('off')            
            cnt += 1

In [ ]:
# train our gan

'''We get a ran-dom mini-batch of MNIST images as real examples and generate a mini-batch of fakeimages
 from random noise vectors z. We then use those to train the Discriminator net-work  while 
keeping  the  Generator’s  parameters  constant.  Next,  we  generate  a  mini-batch of fake 
  images and use those to train the Generator network while keeping theDiscriminator’s parameters fixed.
   We repeat this for each iteration. To generatez, we sample from the standard normal distribution '''

losses = []
accuracies = []
iteration_checkpoints = []


def train(iterations, batch_size, sample_interval):

    (x_train, _), (_, _) = mnist.load_data()
    x_train = x_train/127.5 - 1.0 #rescale grayscale values(1, 255) to [-1, 1]
    x_train = np.expand_dims(x_train, axis = 3)

    real = np.ones((batch_size, 1)) # labels for real images
    fake = np.zeros((batch_size, 1)) # fake images labels

    for iteration in range(iterations):
        idx = np.random.randint(0, x_train.shape[0], batch_size)       #random batch of real images     
        imgs = x_train[idx]

        #batch of fake images 
        z = np.random.normal(0, 1, (batch_size, 100))             
        gen_imgs = generator.predict(z)
        # trains discriminator
        d_loss_real = discriminator.train_on_batch(imgs, real)            
        d_loss_fake = discriminator.train_on_batch(gen_imgs, fake)        
        d_loss, accuracy = 0.5 * np.add(d_loss_real, d_loss_fake)

        # generate batch of fake images
        z = np.random.normal(0, 1, (batch_size, 100))
        # train generators           
        gen_imgs = generator.predict(z)        
        g_loss = gan.train_on_batch(z, real)

        # save losses and accuracies for plotting
        if (iteration + 1) % sample_interval == 0:            
            losses.append((d_loss, g_loss))                    
            accuracies.append(100.0 * accuracy)            
            iteration_checkpoints.append(iteration + 1)

        # output training progress
        print("%d [D loss: %f, acc.: %.2f%%] [G loss: %f]" % (iteration + 1, d_loss, 100.0 * accuracy, g_loss)) 
        # output sample images           
        sample_images(generator)

In [ ]:
iterations = 20000           
batch_size = 128
sample_interval = 1000

train(iterations, batch_size, sample_interval)

1 [D loss: 1.097088, acc.: 30.86%] [G loss: 0.683705]
2 [D loss: 0.877438, acc.: 50.78%] [G loss: 0.688056]
3 [D loss: 0.517644, acc.: 64.06%] [G loss: 0.710748]
4 [D loss: 0.113600, acc.: 99.61%] [G loss: 0.741198]
5 [D loss: 0.042242, acc.: 100.00%] [G loss: 0.762275]
6 [D loss: 0.095548, acc.: 97.27%] [G loss: 0.768528]
7 [D loss: 0.109643, acc.: 96.88%] [G loss: 0.760631]
8 [D loss: 0.048985, acc.: 99.61%] [G loss: 0.749632]
9 [D loss: 0.028780, acc.: 99.61%] [G loss: 0.741368]
10 [D loss: 0.013284, acc.: 100.00%] [G loss: 0.730226]
11 [D loss: 0.014109, acc.: 99.61%] [G loss: 0.720760]
12 [D loss: 0.015090, acc.: 100.00%] [G loss: 0.714766]
13 [D loss: 0.012650, acc.: 100.00%] [G loss: 0.701385]
14 [D loss: 0.011662, acc.: 100.00%] [G loss: 0.696538]
15 [D loss: 0.010940, acc.: 100.00%] [G loss: 0.693448]
16 [D loss: 0.010737, acc.: 100.00%] [G loss: 0.691579]
17 [D loss: 0.006759, acc.: 100.00%] [G loss: 0.690873]
18 [D loss: 0.007796, acc.: 100.00%] [G loss: 0.695606]
19 [D loss

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:14: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  


22 [D loss: 0.009803, acc.: 100.00%] [G loss: 0.777085]
23 [D loss: 0.005595, acc.: 100.00%] [G loss: 0.810905]
24 [D loss: 0.006287, acc.: 100.00%] [G loss: 0.839552]
25 [D loss: 0.006915, acc.: 100.00%] [G loss: 0.862583]
26 [D loss: 0.008919, acc.: 100.00%] [G loss: 0.873566]
27 [D loss: 0.018565, acc.: 99.61%] [G loss: 0.877693]
28 [D loss: 0.018479, acc.: 100.00%] [G loss: 0.887155]
29 [D loss: 0.012471, acc.: 100.00%] [G loss: 0.898683]
30 [D loss: 0.009110, acc.: 100.00%] [G loss: 0.911254]
31 [D loss: 0.009706, acc.: 100.00%] [G loss: 0.918581]
32 [D loss: 0.009989, acc.: 100.00%] [G loss: 0.917851]
33 [D loss: 0.009478, acc.: 100.00%] [G loss: 0.913583]
34 [D loss: 0.010985, acc.: 100.00%] [G loss: 0.912990]
35 [D loss: 0.009762, acc.: 100.00%] [G loss: 0.927585]
36 [D loss: 0.008296, acc.: 100.00%] [G loss: 0.933788]
37 [D loss: 0.009663, acc.: 100.00%] [G loss: 0.930868]
38 [D loss: 0.010780, acc.: 100.00%] [G loss: 0.920524]
39 [D loss: 0.007594, acc.: 100.00%] [G loss: 0.9